In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import wandb
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#############################
# Data Loading & Transforms
#############################

# Load data from .pt files
train_data = torch.load("Q1/train_data.pt")  # Shape: (num_samples, 3, 36, 36)
train_labels = torch.load("Q1/train_labels.pt")
test_data = torch.load("Q1/test_data.pt")
test_labels = torch.load("Q1/test_labels.pt")

# Custom Dataset that applies a transform to the tensor data
class CustomTensorDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        """
        data: Tensor of shape (N, 3, H, W)
        labels: Tensor of shape (N,)
        transform: function to apply to each sample
        """
        self.data = data
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

# Define transforms
# For 36x36 images: convert to float, scale to [0,1], then normalize.
transform36 = transforms.Compose([
    transforms.Lambda(lambda x: x.float() / 255.0),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# For 224x224 images: resize (using bilinear interpolation) then normalize.
def resize_and_normalize(x, size=(224, 224)):
    x = x.float() / 255.0
    x = x.unsqueeze(0)  # (1, 3, 36, 36)
    x = F.interpolate(x, size=size, mode='bilinear', align_corners=False)
    x = x.squeeze(0)    # back to (3, 224, 224)
    x = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])(x)
    return x

# Create dataset objects
train_dataset_36 = CustomTensorDataset(train_data, train_labels, transform=transform36)
test_dataset_36 = CustomTensorDataset(test_data, test_labels, transform=transform36)

train_dataset_224 = CustomTensorDataset(train_data, train_labels, transform=resize_and_normalize)
test_dataset_224 = CustomTensorDataset(test_data, test_labels, transform=resize_and_normalize)

# Create DataLoaders
train_loader_36 = DataLoader(train_dataset_36, batch_size=32, shuffle=True)
test_loader_36 = DataLoader(test_dataset_36, batch_size=32, shuffle=False)
train_loader_224 = DataLoader(train_dataset_224, batch_size=32, shuffle=True)
test_loader_224 = DataLoader(test_dataset_224, batch_size=32, shuffle=False)

#############################
# WandB Initialization & Metric Definitions
#############################


#############################
# Training & Evaluation Functions
#############################

def evaluate_epoch(model, data_loader, criterion, device):
    """Evaluate model on data_loader and return loss and accuracy."""
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    avg_loss = total_loss / len(data_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001, run_name='default', use_scheduler=False):
    """
    Trains a model and evaluates it on the test set after each epoch.
    If use_scheduler is True, uses ReduceLROnPlateau on the test loss.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                               factor=0.1, patience=2)
    
    wandb.init(project='resnet-experiments', reinit=True)
    wandb.define_metric("epoch")
    wandb.define_metric("train_loss", step_metric="epoch")
    wandb.define_metric("train_accuracy", step_metric="epoch")
    wandb.define_metric("test_loss", step_metric="epoch")
    wandb.define_metric("test_accuracy", step_metric="epoch")
    wandb.run.name = run_name
    run = wandb.init(project='resnet-experiments', name=run_name, reinit=True)
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_loss = total_loss / len(train_loader)
        train_accuracy = 100. * correct / total
        
        # Evaluate on test set after each epoch
        test_loss, test_accuracy = evaluate_epoch(model, test_loader, criterion, device)
        
        # Step the scheduler if used
        if scheduler:
            scheduler.step(test_loss)
        
        # Log all metrics once per epoch, including current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "test_loss": test_loss,
            "test_accuracy": test_accuracy,
            "learning_rate": current_lr
        })
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}% | LR: {current_lr}")
    run.finish()
    return model

def evaluate_model(model, test_loader):
    """Evaluate the model and display a classification report and confusion matrix."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    print(classification_report(y_true, y_pred))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

#############################
# Architecture Modifications (for 36x36 input)
#############################

num_classes = int(train_labels.max().item() + 1)  # Assumes labels are 0-indexed

# ---------------------
# Models Trained From Scratch
# ---------------------

# Modification A: Replace conv1 (3×3, stride 1, padding 1) and remove maxpool.
class ModifiedResNet18_A(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_A, self).__init__()
        self.model = resnet18(weights=None)  # training from scratch
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

# Modification B: In addition to A, modify layer2's first block to remove downsampling.
class ModifiedResNet18_B(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_B, self).__init__()
        self.model = resnet18(weights=None)  # training from scratch
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        # Modify layer2's first block
        self.model.layer2[0].conv1.stride = (1, 1)
        if self.model.layer2[0].downsample is not None:
            self.model.layer2[0].downsample[0].stride = (1, 1)
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

# Modification C: In addition to A and B, also modify layer3's first block to remove downsampling.
class ModifiedResNet18_C(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_C, self).__init__()
        self.model = resnet18(weights=None)  # training from scratch
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        # Modification B
        self.model.layer2[0].conv1.stride = (1, 1)
        if self.model.layer2[0].downsample is not None:
            self.model.layer2[0].downsample[0].stride = (1, 1)
        # Modification C
        self.model.layer3[0].conv1.stride = (1, 1)
        if self.model.layer3[0].downsample is not None:
            self.model.layer3[0].downsample[0].stride = (1, 1)
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

# ---------------------
# Pretrained Models with Modifications
# ---------------------

# Pretrained Modification A:
class ModifiedResNet18_Pretrained_A(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_Pretrained_A, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        # Replace conv1 and reinitialize
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        nn.init.kaiming_normal_(self.model.conv1.weight, mode='fan_out', nonlinearity='relu')
        self.model.maxpool = nn.Identity()
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

# Pretrained Modification B:
class ModifiedResNet18_Pretrained_B(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_Pretrained_B, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        nn.init.kaiming_normal_(self.model.conv1.weight, mode='fan_out', nonlinearity='relu')
        self.model.maxpool = nn.Identity()
        self.model.layer2[0].conv1.stride = (1, 1)
        if self.model.layer2[0].downsample is not None:
            self.model.layer2[0].downsample[0].stride = (1, 1)
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

# Pretrained Modification C:
class ModifiedResNet18_Pretrained_C(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18_Pretrained_C, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        nn.init.kaiming_normal_(self.model.conv1.weight, mode='fan_out', nonlinearity='relu')
        self.model.maxpool = nn.Identity()
        self.model.layer2[0].conv1.stride = (1, 1)
        if self.model.layer2[0].downsample is not None:
            self.model.layer2[0].downsample[0].stride = (1, 1)
        self.model.layer3[0].conv1.stride = (1, 1)
        if self.model.layer3[0].downsample is not None:
            self.model.layer3[0].downsample[0].stride = (1, 1)
        self.model.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        return self.model(x)

#############################
# Training and Evaluations
#############################

# --- Training on 36x36 Images ---

# print("=== Training Standard ResNet18 from Scratch on 36x36 ===")
# model_scratch = resnet18(weights=None)
# model_scratch.fc = nn.Linear(512, num_classes)
# # Use scheduler for non-pretrained models (lr=0.001)
# trained_model_scratch = train_model(model_scratch, train_loader_36, test_loader_36,
#                                     epochs=10, lr=0.001, run_name='ResNet18_Scratch_36',
#                                     use_scheduler=True)

# print("=== Training Standard ResNet18 Pretrained on 36x36 ===")
# model_pretrained = resnet18(weights=ResNet18_Weights.DEFAULT)
# model_pretrained.fc = nn.Linear(512, num_classes)
# # Use lower lr (0.0001) for pretrained models; no scheduler.
# trained_model_pretrained = train_model(model_pretrained, train_loader_36, test_loader_36,
#                                        epochs=10, lr=0.0001, run_name='ResNet18_Pretrained_36',
#                                        use_scheduler=False)

# # --- Training on Resized 224x224 Images ---

# print("=== Training Standard ResNet18 from Scratch on 224x224 ===")
# model_scratch_224 = resnet18(weights=None)
# model_scratch_224.fc = nn.Linear(512, num_classes)
# trained_model_scratch_224 = train_model(model_scratch_224, train_loader_224, test_loader_224,
#                                         epochs=10, lr=0.001, run_name='ResNet18_Scratch_224',
#                                         use_scheduler=True)

# print("=== Training Standard ResNet18 Pretrained on 224x224 ===")
# model_pretrained_224 = resnet18(weights=ResNet18_Weights.DEFAULT)
# model_pretrained_224.fc = nn.Linear(512, num_classes)
# trained_model_pretrained_224 = train_model(model_pretrained_224, train_loader_224, test_loader_224,
#                                            epochs=10, lr=0.0001, run_name='ResNet18_Pretrained_224',
#                                            use_scheduler=False)

# # --- Training Modified Architectures (from scratch) on 36x36 ---

# print("=== Training ModifiedResNet18_A (Scratch) ===")
# modelA = ModifiedResNet18_A(num_classes)
# trained_modelA = train_model(modelA, train_loader_36, test_loader_36,
#                              epochs=10, lr=0.001, run_name='Modified_Scratch_A',
#                              use_scheduler=True)

# print("=== Training ModifiedResNet18_B (Scratch) ===")
# modelB = ModifiedResNet18_B(num_classes)
# trained_modelB = train_model(modelB, train_loader_36, test_loader_36,
#                              epochs=10, lr=0.001, run_name='Modified_Scratch_B',
#                              use_scheduler=True)

print("=== Training ModifiedResNet18_C (Scratch) ===")
modelC = ModifiedResNet18_C(num_classes)
trained_modelC = train_model(modelC, train_loader_36, test_loader_36,
                             epochs=10, lr=0.001, run_name='Modified_Scratch_C',
                             use_scheduler=True)

# --- Training Modified Architectures (Pretrained) on 36x36 ---

print("=== Training ModifiedResNet18_Pretrained_A ===")
modelPreA = ModifiedResNet18_Pretrained_A(num_classes)
trained_modelPreA = train_model(modelPreA, train_loader_36, test_loader_36,
                                epochs=10, lr=0.0001, run_name='Modified_Pretrained_A',
                                use_scheduler=False)

print("=== Training ModifiedResNet18_Pretrained_B ===")
modelPreB = ModifiedResNet18_Pretrained_B(num_classes)
trained_modelPreB = train_model(modelPreB, train_loader_36, test_loader_36,
                                epochs=10, lr=0.0001, run_name='Modified_Pretrained_B',
                                use_scheduler=False)

print("=== Training ModifiedResNet18_Pretrained_C ===")
modelPreC = ModifiedResNet18_Pretrained_C(num_classes)
trained_modelPreC = train_model(modelPreC, train_loader_36, test_loader_36,
                                epochs=10, lr=0.0001, run_name='Modified_Pretrained_C',
                                use_scheduler=False)



=== Training ModifiedResNet18_C (Scratch) ===


wandb: Currently logged in as: siddharth-singh2504 (siddharth-singh2504-iiit-hyderabad). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1: Train Loss: 1.4596, Train Acc: 46.18% | Test Loss: 1.1201, Test Acc: 59.57% | LR: 0.001
Epoch 2: Train Loss: 1.0321, Train Acc: 63.13% | Test Loss: 0.9603, Test Acc: 66.30% | LR: 0.001
Epoch 3: Train Loss: 0.8482, Train Acc: 70.06% | Test Loss: 0.8312, Test Acc: 71.38% | LR: 0.001
Epoch 4: Train Loss: 0.7080, Train Acc: 75.34% | Test Loss: 0.7404, Test Acc: 73.93% | LR: 0.001
Epoch 5: Train Loss: 0.6071, Train Acc: 79.14% | Test Loss: 0.6810, Test Acc: 76.49% | LR: 0.001
Epoch 6: Train Loss: 0.5259, Train Acc: 81.84% | Test Loss: 0.5956, Test Acc: 79.83% | LR: 0.001
Epoch 7: Train Loss: 0.4614, Train Acc: 83.97% | Test Loss: 0.5038, Test Acc: 82.82% | LR: 0.001
Epoch 8: Train Loss: 0.4029, Train Acc: 86.15% | Test Loss: 0.5872, Test Acc: 81.08% | LR: 0.001
Epoch 9: Train Loss: 0.3521, Train Acc: 87.82% | Test Loss: 0.4624, Test Acc: 84.52% | LR: 0.001
Epoch 10: Train Loss: 0.3082, Train Acc: 89.35% | Test Loss: 0.5299, Test Acc: 82.94% | LR: 0.001


epoch,▁▂▃▃▄▅▆▆▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▃▄▅▆▇█▇██
test_loss,█▆▅▄▃▂▁▂▁▂
train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
epoch,10
learning_rate,0.001
test_accuracy,82.94
test_loss,0.52993
train_accuracy,89.346


=== Training ModifiedResNet18_Pretrained_A ===


...

Epoch 1: Train Loss: 0.6366, Train Acc: 78.21% | Test Loss: 0.3855, Test Acc: 86.82% | LR: 0.0001
Epoch 2: Train Loss: 0.2743, Train Acc: 90.64% | Test Loss: 0.2971, Test Acc: 89.85% | LR: 0.0001
Epoch 3: Train Loss: 0.1471, Train Acc: 95.04% | Test Loss: 0.2867, Test Acc: 90.27% | LR: 0.0001
Epoch 4: Train Loss: 0.0915, Train Acc: 96.99% | Test Loss: 0.2808, Test Acc: 90.84% | LR: 0.0001
Epoch 5: Train Loss: 0.0657, Train Acc: 97.83% | Test Loss: 0.2971, Test Acc: 90.95% | LR: 0.0001
Epoch 6: Train Loss: 0.0539, Train Acc: 98.22% | Test Loss: 0.3345, Test Acc: 89.95% | LR: 0.0001
Epoch 7: Train Loss: 0.0453, Train Acc: 98.54% | Test Loss: 0.3161, Test Acc: 90.67% | LR: 0.0001
Epoch 8: Train Loss: 0.0406, Train Acc: 98.67% | Test Loss: 0.3154, Test Acc: 91.19% | LR: 0.0001
Epoch 9: Train Loss: 0.0393, Train Acc: 98.65% | Test Loss: 0.3092, Test Acc: 91.48% | LR: 0.0001
Epoch 10: Train Loss: 0.0339, Train Acc: 98.89% | Test Loss: 0.3339, Test Acc: 91.12% | LR: 0.0001


epoch,▁▂▃▃▄▅▆▆▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▆▇▇▆▇██▇
test_loss,█▂▁▁▂▅▃▃▃▅
train_accuracy,▁▅▇▇██████
train_loss,█▄▂▂▁▁▁▁▁▁
epoch,10
learning_rate,0.0001
test_accuracy,91.12
test_loss,0.33388
train_accuracy,98.886


=== Training ModifiedResNet18_Pretrained_C ===


Epoch 1: Train Loss: 0.7885, Train Acc: 72.96% | Test Loss: 0.5317, Test Acc: 81.74% | LR: 0.0001
Epoch 2: Train Loss: 0.4013, Train Acc: 86.49% | Test Loss: 0.3815, Test Acc: 87.15% | LR: 0.0001
Epoch 3: Train Loss: 0.2598, Train Acc: 91.29% | Test Loss: 0.3568, Test Acc: 87.98% | LR: 0.0001
Epoch 4: Train Loss: 0.1714, Train Acc: 94.43% | Test Loss: 0.3311, Test Acc: 88.87% | LR: 0.0001
Epoch 5: Train Loss: 0.1184, Train Acc: 96.16% | Test Loss: 0.3453, Test Acc: 88.98% | LR: 0.0001
Epoch 6: Train Loss: 0.0866, Train Acc: 97.36% | Test Loss: 0.3324, Test Acc: 89.58% | LR: 0.0001
Epoch 7: Train Loss: 0.0711, Train Acc: 97.76% | Test Loss: 0.3759, Test Acc: 88.63% | LR: 0.0001
Epoch 8: Train Loss: 0.0642, Train Acc: 97.90% | Test Loss: 0.3575, Test Acc: 89.32% | LR: 0.0001
Epoch 9: Train Loss: 0.0547, Train Acc: 98.23% | Test Loss: 0.3715, Test Acc: 89.25% | LR: 0.0001
Epoch 10: Train Loss: 0.0468, Train Acc: 98.50% | Test Loss: 0.3772, Test Acc: 89.25% | LR: 0.0001


epoch,▁▂▃▃▄▅▆▆▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▆▇▇▇█▇███
test_loss,█▃▂▁▁▁▃▂▂▃
train_accuracy,▁▅▆▇▇█████
train_loss,█▄▃▂▂▁▁▁▁▁
epoch,10
learning_rate,0.0001
test_accuracy,89.25
test_loss,0.37722
train_accuracy,98.504


# Analysis
- larger 224 x 224 images need longer compute as they fill up the gpu storage faster but have better results
- pretrained models perform better than non pretrained ones
- altering kernel sizes/ maxpool mitigate diff b/w model performance on 36 x 36 vs 224 x 224 images
- all plots in /q1